In [ ]:
##Instalação bibliotecas
!pip install geemap
!pip install earth-engine api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement earth-engine (from versions: none)
ERROR: No matching distribution found for earth-engine


In [ ]:
##Importar bibliotecas
import ee
import geemap

In [ ]:
#Autenticação
ee.Authenticate()
ee.Initialize(project='ee-hzotelli')

In [ ]:
# Criar um mapa interativo
Map = geemap.Map()

# Carregar o shapefile (Área de Estudo)
shapefile = ee.FeatureCollection('projects/ee-hzotelli/assets/loc_apyterewa') #Carrega um shapefile da pasta de assets no Earth Engine
area = shapefile.geometry() #Extrai apenas a geometria da área

# Carregar coleção Landsat 8
image_collection = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterDate('2024-01-01', '2024-12-31')
    .filterBounds(area)
    .filter(ee.Filter.inList('WRS_PATH', [225])) #Retirar cenas que peguem a "beirada" da TI
    .filter(ee.Filter.inList('WRS_ROW', [64]))
    .sort('system:time_start') #Ordem cronológica
)

print("Número de imagens na coleção:", image_collection.size().getInfo())


Número de imagens na coleção: 70


In [ ]:
# Função para mascarar nuvens e aerossóis
def mask_clouds(image):
    qa = image.select('QA_PIXEL')
    cloud_bit = 1 << 3 #Nuvens
    shadow_bit = 1 << 4 #Sombras
    cirrus_bit = 1 << 2 #Cirrus

    cloud_mask = (
        qa.bitwiseAnd(cloud_bit).eq(0)
        .And(qa.bitwiseAnd(shadow_bit).eq(0))
        .And(qa.bitwiseAnd(cirrus_bit).eq(0))
    )

    aerosol = image.select("SR_QA_AEROSOL").bitwiseAnd(0b11000000).rightShift(6)
    aerosol_mask = aerosol.lte(1)

# Aplicar buffer nas bordas da nuvem
    combined_mask = cloud_mask.And(aerosol_mask)
    buffered_mask = combined_mask.focal_min(radius=100, units='meters')

    return image.updateMask(buffered_mask) #'focal_min' para expandir o buffer das nuvens (evita bordas falsas)

# Aplicar a máscara de nuvens na coleção
masked_collection = image_collection.map(mask_clouds) #Aplica a função de máscara a todas as imagens da coleção

print("Número de imagens após a máscara de nuvens:", masked_collection.size().getInfo())

# Função para preencher pixels nublados de forma independente
def fill_clouds_per_pixel(image):
    current_time = image.get('system:time_start') #Pega a data da imagem atual
    next_images = masked_collection.filter(ee.Filter.gt('system:time_start', current_time)) #Seleciona apenas as imagens posteriores na coleção

    mask = image.mask().Not() #Cria uma máscara que identifica os pixels que estão vazios (ou seja, foram mascarados por nuvens ou aerossóis)
    filled_image = image #Começa com a imagem original

    def replace_pixel(next_img, prev_filled): #usada repetidamente sobre as imagens seguintes para preencher os buracos da imagem atual
        prev_filled = ee.Image(prev_filled) #imagem original
        next_img = ee.Image(next_img) #imagem seguinte
        return prev_filled.unmask(next_img.updateMask(mask))
        #máscara invertida: pegar apenas os pixels válidos da next_img onde a prev_filled está com nuvem

    filled_image = ee.Image(next_images.iterate(replace_pixel, filled_image)) # GEE itera (loopa) sobre todas as imagens posteriores (next_images)
    return filled_image

# Aplicar o preenchimento pixel a pixel na coleção inteira
filled_collection = masked_collection.map(fill_clouds_per_pixel)

print("Número de imagens após substituição de pixels nublados:", filled_collection.size().getInfo())

# Função para calcular o NDVI e aplicar o fator de escala
def scale_and_ndvi(image):
    # Verificar se a imagem tem system:time_start (dowload)
    time_start = ee.Algorithms.If(
        image.propertyNames().contains('system:time_start'),
        image.get('system:time_start'),
        ee.Date('1970-01-01').millis()  # Valor padrão se não existir
    )

    # Aplicar o fator de escala
    optical_bands = image.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(0.0000275).add(-0.2)

    # Calcular NDVI
    ndvi = optical_bands.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

    # Adicionar bandas originais (escaladas) e NDVI
    return optical_bands.addBands(ndvi).set({
        'date': ee.Date(time_start).format('YYYY-MM-dd')
    })

# Aplicar o cálculo de NDVI e fator de escala na coleção
processed_collection = filled_collection.map(scale_and_ndvi)

print("Número de imagens na coleção pós-processamento (NDVI calculado):", processed_collection.size().getInfo())

# Selecionar imagens para visualização
image_index = 1  # Você pode alterar este índice

original_masked_sample = ee.Image(masked_collection.toList(masked_collection.size()).get(image_index))
processed_sample = ee.Image(processed_collection.toList(processed_collection.size()).get(image_index))
original_image = ee.Image(image_collection.toList(image_collection.size()).get(image_index))

# Adicionar camadas ao mapa

# Área de estudo
Map.addLayer(area, {'color': 'gray'}, 'Área de Estudo')

# Imagem original (sem máscara)
Map.addLayer(original_image.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 7787.0, 'max': 24259.0}, 'Imagem Original (RGB)')

# Imagem original pós-máscara (pixels válidos)
Map.addLayer(original_masked_sample.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 8100.0, 'max': 14474.0}, 'Imagem Original Pós Máscara')

# Imagem processada (preenchida)
Map.addLayer(processed_sample.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 0.0, 'max': 0.4}, 'Imagem Processada (Preenchida)')

# NDVI (precisa de min/max para cores)
vis_params_ndvi = {
    'bands': ['NDVI'],
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}
Map.addLayer(processed_sample.select('NDVI'), vis_params_ndvi, 'NDVI')

# Centralizar e mostrar mapa
Map.centerObject(area, zoom=10)
Map.addLayerControl()
Map

Número de imagens após a máscara de nuvens: 70
Número de imagens após substituição de pixels nublados: 70
Número de imagens na coleção pós-processamento (NDVI calculado): 70


Map(center=[-5.636787979328296, -52.03658924055257], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# EXPORTAÇÃO DAS IMAGENS
# --- Exportar RGB de TODAS ---
def export_image(image, filename, folder, bands, scale=30):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=filename,
        folder=folder,
        fileNamePrefix=filename.replace('/', '_').replace(':', '_'),
        region=area,
        scale=scale,
        maxPixels=1e13,
        crs='EPSG:4326',
        fileFormat='GeoTIFF',
        formatOptions={'cloudOptimized': True}
    )
    task.start()

folder_name = "Landsat_Processed"
image_list = processed_collection.toList(processed_collection.size())

for i in range(0, processed_collection.size().getInfo()):
    img = ee.Image(image_list.get(i))
    date_str = img.get('date').getInfo().replace(':', '-').replace(' ', '_').replace('/', '-')

    export_image(
        img.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']),
        f'L8_BANDS_{date_str}',
        folder_name,
        ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']
    )

print("Exportação de bandas SR_B* iniciada para todas as imagens.")


In [ ]:
# --- Exportar UMA imagem de NDVI por data ---

# Escolha a data desejada (formato 'YYYY-MM-DD')
data_escolhida = '2023-07-21'

# Filtrar pela propriedade 'date' atribuída durante o processamento
ndvi_img = processed_collection.filter(ee.Filter.eq('date', data_escolhida)).first()

# Função para exportar a imagem
def export_image(image, description, folder, bands):
    task = ee.batch.Export.image.toDrive(
        image=image.select(bands),
        description=description,
        folder=folder,
        scale=30,
        region=area,
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    task.start()

# Verificar se a imagem existe antes de exportar
info = ndvi_img.getInfo()
if info:
    nome_data = ndvi_img.get('date').getInfo().replace(':', '-').replace(' ', '_').replace('/', '-')
    export_image(
        ndvi_img.select('NDVI'),
        f'NDVI_{nome_data}',
        'NDVI_Export',
        ['NDVI']
    )
    print(f"Exportação iniciada para a imagem de {data_escolhida}")
else:
    print("Nenhuma imagem encontrada para essa data.")